In [1]:
import findspark

findspark.init()

ModuleNotFoundError: No module named 'findspark'

In [ ]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm").load("iris-train.txt")

In [ ]:
# Split the training data into train and test for parameter tuning
splits = data.randomSplit([0.6, 0.4], 1)
train = splits[0]
test = splits[1]

bestAccuracy = 0
for i in range(10,51,10):
    for t in [1E-5,1E-6,1E-7]:
        for f in [True,False]:
            # instantiate the base classifier.
            lr = LogisticRegression(maxIter=i, tol=t, fitIntercept=f)

            # instantiate the One Vs Rest Classifier.
            ovr = OneVsRest(classifier=lr)

            # train the multiclass model.
            ovrModel = ovr.fit(train)
            
            # compute accuracy on the test set
            result = ovrModel.transform(test)
            predictionAndLabels = result.select("prediction", "label")
            evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
            accuracy = evaluator.evaluate(predictionAndLabels)
            
            if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestI = i
                bestT = t
                bestF = f

In [ ]:
print("Best accuracy = " + str(bestAccuracy) + " obtained with parameters maxIter = " + str(bestI) + " tol = " + str(bestT) + " and fitIntercept = " + str(bestF))

In [ ]:
result.show(result.count(), truncate=25)